### Angle 1

import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge

def calculate_directions(positions):
    velocities = positions[1:] - positions[:-1]
    if velocities.size(0) < positions.size(0):
        velocities = torch.cat([velocities, velocities[-1:].repeat(1, 1, 1)], dim=0)
    angles = torch.atan2(velocities[:, :, 1], velocities[:, :, 0])
    return angles

def angle_difference(a, b):
    diff = torch.atan2(torch.sin(a - b), torch.cos(a - b))
    return diff

def filter_neighbors(positions, angles, view_angle=np.pi/2, tolerance=1e-2):
    T, N, _ = positions.shape
    adjacency_matrices = torch.zeros(T, N, N, dtype=torch.float)
    for t in range(T):
        for i in range(N):
            direction_i = angles[t, i]
            for j in range(N):
                if i != j:
                    other_pos = positions[t, j]
                    relative_position = other_pos - positions[t, i]
                    angle_to_other = torch.atan2(relative_position[1], relative_position[0])
                    angle_diff = angle_difference(angle_to_other, direction_i)
                    if -view_angle - tolerance <= angle_diff <= view_angle + tolerance:
                        adjacency_matrices[t, i, j] = 1
    return adjacency_matrices

def visualize_positions_with_view_cone(t, i, positions, adjacency_matrix, angles, view_angle=np.pi):
    fig, ax = plt.subplots()
    ax.scatter(positions[t, :, 0], positions[t, :, 1], color='blue', label='All pedestrians')
    ax.scatter(positions[t, i, 0], positions[t, i, 1], color='red', label='Selected pedestrian')

    direction = angles[t, i].item()
    radius = 10
    wedge = Wedge((positions[t, i, 0], positions[t, i, 1]), radius, 
                  np.degrees(direction - view_angle), np.degrees(direction + view_angle),
                  color='red', alpha=0.2)
    ax.add_patch(wedge)
    ax.arrow(positions[t, i, 0], positions[t, i, 1], 
             np.cos(direction), np.sin(direction), 
             head_width=0.2, head_length=0.3, fc='red', ec='red')

    for j in range(len(adjacency_matrix[t, i])):
        if adjacency_matrix[t, i, j] == 1:
            ax.scatter(positions[t, j, 0], positions[t, j, 1], color='green', label='In view angle')

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title(f'Visualization at time step {t} for pedestrian {i}')
    plt.grid(True)
    plt.axis('equal')
    plt.show()

# 生成数据
positions = torch.tensor(np.random.rand(5, 7, 2) * 10, dtype=torch.float32)  # T x N x 2
angles = calculate_directions(positions)
adjacency_matrices = filter_neighbors(positions, angles)

# 可视化第3个时间步的第1个行人的视角
visualize_positions_with_view_cone(0, 4, positions, adjacency_matrices, angles, view_angle=np.pi/3)


### Angle 2

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Wedge

def calculate_directions(positions):
    velocities = positions[1:] - positions[:-1]
    if velocities.size(0) < positions.size(0):
        velocities = torch.cat([velocities, velocities[-1:].repeat(1, 1, 1)], dim=0)
    angles = torch.atan2(velocities[:, :, 1], velocities[:, :, 0])
    return angles

def filter_neighbors(positions, angles, view_angle=np.pi/2, tolerance=1e-2):
    T, N, _ = positions.shape
    adjacency_matrices = torch.zeros(T, N, N, dtype=torch.float)
    for t in range(T):
        for i in range(N):
            direction_i = angles[t, i]
            # 行人 i 的朝向向量
            heading_vector = torch.tensor([torch.cos(direction_i), torch.sin(direction_i)])
            for j in range(N):
                if i != j:
                    relative_position = positions[t, j] - positions[t, i]
                    # 计算有符号角度
                    cross_prod = heading_vector[0] * relative_position[1] - heading_vector[1] * relative_position[0]
                    dot_prod = heading_vector[0] * relative_position[0] + heading_vector[1] * relative_position[1]
                    angle_to_other = torch.atan2(cross_prod, dot_prod)
                    # 判断是否在视野范围内
                    if -view_angle - tolerance <= angle_to_other <= view_angle + tolerance:
                        adjacency_matrices[t, i, j] = 1
    return adjacency_matrices

def visualize_positions_with_view_cone(t, i, positions, adjacency_matrix, angles, view_angle=np.pi):
    fig, ax = plt.subplots()
    ax.scatter(positions[t, :, 0], positions[t, :, 1], color='blue', label='All pedestrians')
    ax.scatter(positions[t, i, 0], positions[t, i, 1], color='red', label='Selected pedestrian')

    direction = angles[t, i].item()
    radius = 10
    wedge = Wedge((positions[t, i, 0], positions[t, i, 1]), radius, 
                  np.degrees(direction - view_angle), np.degrees(direction + view_angle),
                  color='red', alpha=0.2)
    ax.add_patch(wedge)
    ax.arrow(positions[t, i, 0], positions[t, i, 1], 
             np.cos(direction), np.sin(direction), 
             head_width=0.2, head_length=0.3, fc='red', ec='red')

    for j in range(len(adjacency_matrix[t, i])):
        if adjacency_matrix[t, i, j] == 1:
            ax.scatter(positions[t, j, 0], positions[t, j, 1], color='green', label='In view angle')

    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.legend(by_label.values(), by_label.keys())
    plt.xlabel('X Position')
    plt.ylabel('Y Position')
    plt.title(f'Visualization at time step {t} for pedestrian {i}')
    plt.grid(True)
    plt.axis('equal')
    plt.show()

# 生成数据
positions = torch.tensor(np.random.rand(5, 7, 2) * 10, dtype=torch.float32)  # T x N x 2
angles = calculate_directions(positions)
adjacency_matrices = filter_neighbors(positions, angles)

# 可视化第1个时间步的第5个行人的视角
visualize_positions_with_view_cone(0, 4, positions, adjacency_matrices, angles, view_angle=np.pi/3)
